In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import nltk

os.chdir('/home/jovyan/shared/2020_06_10_bad_reviewer')

## Get submissions

In [2]:
# All texts that were parsed
submissions_df = pd.read_csv('bin/submissions_df.csv', index_col=[0])

# This marks exactly identical texts as duplicates
# We may want to go further and look for similar texts using some heuristic
assert not submissions_df.duplicated(subset=['uva_peer_assignments_user_id'], keep=False).all() # There are no duplicate user_ids
submissions_df['is_duplicate'] = submissions_df.duplicated(subset=['text'], keep=False) # All duplicate texts must therefore come from different users

print(f'We have {submissions_df[submissions_df.has_post & ~submissions_df.is_duplicate].shape[0]} submissions by users with posts and not also submitted by another user.')
print('This is based on the files that were parsed, of course.')

# Get simple token count
# submissions_df['word_count'] = submissions_df.text.str.split().str.len()

# Get NLTK word count
## This takes a minute because pandas apply just runs on one core
## We could multithread this using Dask, but it's not worth the trouble here
submissions_df['text'] = submissions_df.text.apply(lambda x: nltk.word_tokenize(x) if pd.notnull(x) else [])
submissions_df['word_count'] = submissions_df.text.str.len()

submissions_df.describe()

We have 2735 submissions by users with posts and not also submitted by another user.
This is based on the files that were parsed, of course.


,peer_submission_score,num_skips,word_count
count,27909.000000,27909.000000,27909.000000
mean,16.593088,0.014726,677.555484
std,1.889616,0.128517,286.356858
min,6.000000,0.000000,35.000000
25%,16.000000,0.000000,485.000000
50%,18.000000,0.000000,652.000000
75%,18.000000,0.000000,844.000000
max,18.000000,3.000000,4041.000000


## Get reviews

In [3]:
reviews_df = pd.read_csv('bin/reviews_df.csv', index_col=[0])

## Takes a minute
reviews_df['peer_review_part_free_response_text'] = reviews_df['peer_review_part_free_response_text'].apply(lambda x: nltk.word_tokenize(x) if pd.notnull(x) else [])
display(reviews_df)

,peer_review_id,uva_peer_assignments_user_id,peer_review_created_ts,peer_submission_id,peer_assignment_review_schema_part_prompt_score,peer_assignment_review_schema_part_option_score,peer_assignment_review_schema_part_prompt_free_response,peer_review_part_free_response_text
0,--1K-8NvEeqUARLn1lBFnw,b3a960970d72a6ce0b00c0bdfbccd943b9b64b6a,2020-07-11 12:13:52.365,tyljI8NkEeql1A5WxvNUrQ,Challenge (score),3.0,Feedback (ungraded):The submission’s challenge...,"[search, for, higher, ground]"
1,--1K-8NvEeqUARLn1lBFnw,b3a960970d72a6ce0b00c0bdfbccd943b9b64b6a,2020-07-11 12:13:52.365,tyljI8NkEeql1A5WxvNUrQ,Selection (score),3.0,Feedback (ungraded):The submission’s tool sele...,"[removing, barriers]"
2,--1K-8NvEeqUARLn1lBFnw,b3a960970d72a6ce0b00c0bdfbccd943b9b64b6a,2020-07-11 12:13:52.365,tyljI8NkEeql1A5WxvNUrQ,Application (score),2.0,Feedback (ungraded):The submission’s tool appl...,"[understanding, it]"
3,--1K-8NvEeqUARLn1lBFnw,b3a960970d72a6ce0b00c0bdfbccd943b9b64b6a,2020-07-11 12:13:52.365,tyljI8NkEeql1A5WxvNUrQ,Insight (score),3.0,Feedback (ungraded):The submission’s insight d...,"[drilling, down, to, the, essence]"
4,--1K-8NvEeqUARLn1lBFnw,b3a960970d72a6ce0b00c0bdfbccd943b9b64b6a,2020-07-11 12:13:52.365,tyljI8NkEeql1A5WxvNUrQ,Approach (score),3.0,Feedback (ungraded):The submission’s approach ...,"[increasing, speed, of, learning]"
...,...,...,...,...,...,...,...,...
1084652,zztv8lPNEei7NQqkzSZOZA,da809ddc4f894b61d858fa3a38f79aad347c5ddc,2018-05-09 21:13:24.498,L7swSFLyEei3JxKjZPnMdA,Application (score),1.0,Feedback (ungraded): The submission’s tool app...,"[Only, went, into, detailing, about, storytell..."
1084653,zztv8lPNEei7NQqkzSZOZA,da809ddc4f894b61d858fa3a38f79aad347c5ddc,2018-05-09 21:13:24.498,L7swSFLyEei3JxKjZPnMdA,Insight (score),2.0,Feedback (ungraded): The submission’s insight ...,[]
1084654,zztv8lPNEei7NQqkzSZOZA,da809ddc4f894b61d858fa3a38f79aad347c5ddc,2018-05-09 21:13:24.498,L7swSFLyEei3JxKjZPnMdA,Approach (score),1.0,Feedback (ungraded): The submission’s approach...,"[Mentions, visualization, but, not, how, they,..."
1084655,zztv8lPNEei7NQqkzSZOZA,da809ddc4f894b61d858fa3a38f79aad347c5ddc,2018-05-09 21:13:24.498,L7swSFLyEei3JxKjZPnMdA,Organization (score),2.0,Feedback (ungraded): The submission’s overall ...,"[Developing, the, challenge, further, and, cle..."


## Get Reviewer Stats

In [4]:
def ttr(value):
    # input is a GroupBy object of token sequences
    # we want one long sequence of tokens for each user
    flat_list = [tok for toks in value for tok in toks]

    # check for zero division and return 
    if (seq_len := len(flat_list)) > 0:
        return len(set(flat_list)) / seq_len
    else:
        return 0

reviewer_df = (reviews_df
               .groupby('uva_peer_assignments_user_id')
               .agg(
                   ttr=pd.NamedAgg(column="peer_review_part_free_response_text", aggfunc=ttr),
                   sd=pd.NamedAgg(column="peer_assignment_review_schema_part_option_score", aggfunc='std')
               ))
display(reviewer_df)

,ttr,sd
uva_peer_assignments_user_id,,
0000e5af02da0c7575b3ebd346b55b29f959e90f,0.484185,0.832352
0002c0f31f5c8456bd360dfcd089a64f444e2de0,0.464497,0.685994
00030b378ea62d60a177113b7854eb26cc29e1a9,0.413333,0.832352
000458f7d47a0b6414f9146258829170ae3ed6a9,0.047619,0.000000
00069909160c6bcd9836cdb23e35b1fdaf56d0c3,0.333333,0.383482
...,...,...
fff56a1858c62540bd76bad23db07a2fbfa963fe,0.487719,0.900254
fff7364558963fb9fa218f2fa08d5d2767992207,0.528662,0.460889
fff883a717a37b5699e5105a96d61dc10812cabe,0.071429,0.000000


## Calculate Scores with Parameters

In [5]:
def recalculate_scores(select_submissions, ttr_tolerance=0, sd_tolerance=0):
    return (reviewer_df[(reviewer_df.ttr
                        >=
                        ttr_tolerance)
                        &
                        (reviewer_df.sd
                        >=
                        sd_tolerance)]
            .join(reviews_df
                 .set_index('uva_peer_assignments_user_id'))
            .groupby(['peer_review_id', 'peer_submission_id'])
            .agg({'peer_assignment_review_schema_part_option_score': 'sum'})
            .groupby('peer_submission_id')
            .agg({'peer_assignment_review_schema_part_option_score': 'mean'})
            .reindex(pd.Index(select_submissions.peer_submission_id))
            .set_index(select_submissions.index)
            .peer_assignment_review_schema_part_option_score
            .dropna())

## Make an Interactive Plot

Not sure if this is useful at all, but it was fun to make.

In [6]:
import plotly.graph_objects as go
import statsmodels.api as sm
from ipywidgets import widgets

ModuleNotFoundError: No module named 'plotly'

In [ ]:
sd_tolerance = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=0.5,
    step=0.1,
    description='SD:',
    continuous_update=True
)

ttr_tolerance = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=0.5,
    step=0.1,
    description='TTR:',
    continuous_update=True
)

### TODO: add intslider for num_reviewers required to calculate score

omit_duplicates = widgets.Checkbox(
    description='Omit Duplicates',
    value=False,
)

posters_only = widgets.Checkbox(
    description='Forum Participants Only',
    value=False,
)

container1 = widgets.HBox(children=[omit_duplicates, posters_only])
container2 = widgets.HBox(children=[sd_tolerance, ttr_tolerance])

# Coursera calculates peer_submission_score differently,
# See getting_peer_review_information.ipynb
# We initialize the graph with Coursera scores,
# But these will change after the graph updates.
X = submissions_df['word_count']
Y = submissions_df['peer_submission_score']
best_fit = sm.OLS(Y, sm.add_constant(X)).fit().fittedvalues

# Assign an empty figure widget with two traces
# Scattergl makes it render using WebGL, in your browser
trace1 = go.Scattergl(name='ols', x=X, y=best_fit, mode='lines')
trace2 = go.Scattergl(name='scores', x=X, y=Y.values, mode='markers')

### TODO: Add trace for num essays remaining

### TODO: Add Mouse-Hover for R-squared value of OLS line

g = go.FigureWidget(data=[trace1
                          # , trace2],
                         ],
                    layout=go.Layout(
                        title='Relationship Between Word Count and Calculated Score',
                        xaxis={
                            'range':[50, 1000],
                            'title':'Word Count'
                        },
                        yaxis={
                            'range':[6, 18],
                            'title':'Score'
                        }
                    ))

In [ ]:
def response(change):
    selected_data = submissions_df
    if omit_duplicates.value:
        selected_data = selected_data[~selected_data.is_duplicate]
    if posters_only.value:
        selected_data = selected_data[selected_data.has_post]
        
    Y2 = recalculate_scores(select_submissions=selected_data,
                            ttr_tolerance=ttr_tolerance.value,
                            sd_tolerance=sd_tolerance.value)
    X2 = X.reindex_like(Y2)
    best_fit = sm.OLS(Y2, sm.add_constant(X2)).fit().fittedvalues
    
    with g.batch_update():
        g.data[0].x = X2
        g.data[0].y = best_fit # Regression
        # g.data[1].x = X2
        # g.data[1].y = Y2.values # Scatters
        
sd_tolerance.observe(response, names="value")
ttr_tolerance.observe(response, names="value")
omit_duplicates.observe(response, names="value")
posters_only.observe(response, names="value")

In [ ]:
widgets.VBox([container1,
              container2,
              g])